# Basic Examples for SKlearn Prepackaged Server train with DVC and deployed to MinIO


## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb) to setup Seldon Core with an ingress.

## Get MinIO CLI client tool

Using go get:
```bash   
GO111MODULE=on go get github.com/minio/mc
```

Or follow steps relevant to your platform from official [documentation](https://docs.min.io/docs/minio-client-quickstart-guide.html).

## Install MinIO in cluster

### Helm install minio

In [1]:
%%bash
kubectl create ns minio-system
helm install minio stable/minio \
    --set accessKey=minioadmin \
    --set secretKey=minioadmin \
    --namespace minio-system

namespace/minio-system created
NAME: minio
LAST DEPLOYED: Tue Apr 21 12:20:17 2020
NAMESPACE: minio-system
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Minio can be accessed via port 9000 on the following DNS name from within your cluster:
minio.minio-system.svc.cluster.local

To access Minio from localhost, run the below commands:

  1. export POD_NAME=$(kubectl get pods --namespace minio-system -l "release=minio" -o jsonpath="{.items[0].metadata.name}")

  2. kubectl port-forward $POD_NAME 9000 --namespace minio-system

Read more about port forwarding here: http://kubernetes.io/docs/user-guide/kubectl/kubectl_port-forward/

You can now access Minio server on http://localhost:9000. Follow the below steps to connect to Minio server with mc client:

  1. Download the Minio mc client - https://docs.minio.io/docs/minio-client-quickstart-guide

  2. mc config host add minio-local http://localhost:9000 minioadmin minioadmin S3v4

  3. mc ls minio-local

Alternately, you can use your

In [2]:
!kubectl rollout status deployment -n minio-system minio 

Waiting for deployment "minio" rollout to finish: 0 of 1 updated replicas are available...
deployment "minio" successfully rolled out


### port-forward Minio to localhost

in separate terminal:

```bash
kubectl port-forward -n minio-system svc/minio 9000:9000
```

or follow instructions printed by helm

### Configure mc client

In [3]:
%%bash
mc config host add minio-local http://localhost:9000 minioadmin minioadmin

Added `minio-local` successfully.


## Get DVC CLI tool

Using pip
```bash
pip install --user dvc
```

Or follow steps relevant to your platform from official [documentation](https://dvc.org/doc/install)

## Train model

Key points of training are defined in the following Makefile

In [4]:
!pygmentize Makefile

env:
	python3 -m venv .env
	./.env/bin/pip install --upgrade pip setuptools
	./.env/bin/pip install -r requirements.txt

train:
	.env/bin/python train_iris.py

model: env train


which creates a python environment `.env` and call following training script:

In [5]:
!pygmentize train_iris.py

import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import datasets


def main():
    clf = LogisticRegression(solver="liblinear", multi_class='ovr')
    p = Pipeline([("clf", clf)])
    print("Training model...")
    p.fit(X, y)
    print("Model trained!")

    filename_p = "model.joblib"
    print("Saving model in %s" % filename_p)
    joblib.dump(p, filename_p)
    print("Model saved!")


if __name__ == "__main__":
    print("Loading iris data set...")
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    print("Dataset loaded!")
    main()


### Initial model training (first run)

First training with dvc will crate the `model.dvc` file specifying the hash of output.
We will use that hash to version our model.

In [9]:
%%bash
dvc run -f model.dvc \
          -d Makefile -d requirements.txt -d train_iris.py \
          -o model.joblib \
          --overwrite-dvcfile \
          make model

Running command:
	make model
python3 -m venv .env
./.env/bin/pip install --upgrade pip setuptools
  Using cached https://files.pythonhosted.org/packages/54/0c/d01aa759fdc501a58f431eb594a17495f15b88da142ce14b5845662c13f3/pip-20.0.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a0/df/635cdb901ee4a8a42ec68e480c49f85f4c59e8816effbf57d9e6ee8b3588/setuptools-46.1.3-py3-none-any.whl
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3
  Found existing installation: setuptools 41.2.0
    Uninstalling setuptools-41.2.0:
      Successfully uninstalled setuptools-41.2.0
./.env/bin/pip install -r requirements.txt
  Using cached scikit_learn-0.20.3-cp37-cp37m-manylinux1_x86_64.whl (5.4 MB)
  Using cached numpy-1.18.3-cp37-cp37m-manylinux1_x86_64.whl (20.2 MB)
  Using cached joblib-0.14.1-py2.py3-none-any.whl (294 kB)
  Using cached scipy-1.4.1-cp37-cp37m-manylinux1_x86_64.whl (26.1 MB)
.env/bin/python train_ir

In [10]:
!cat model.dvc

md5: 99124ac10a601ab1d9f07f9c392b5d89
cmd: make model
deps:
- md5: 65fd61883993b68d1937bdc36c59b20c
  path: Makefile
- md5: 7e8ce9f96492fee21db6a59c2b52f34d
  path: requirements.txt
- md5: 49c19c3ea9deb642066c0a457181cfbf
  path: train_iris.py
outs:
- md5: 8104914e6936da9864603b9bc4be2114
  path: model.joblib
  cache: true
  metric: false
  persist: false


The hash of the output is `8104914e6936da9864603b9bc4be2114`

### Reproducing results (successive later runs)

With DVC it is possible to repeat training in reproducible way as versions (hashes) of dependencies are all stored in the `model.dvc` file

In [11]:
%%bash
rm model.joblib -f
dvc repro model.dvc

Running command:
	make model
Output 'model.joblib' didn't change. Skipping saving.

To track the changes with git, run:

	git add model.dvc


## Add trained model to remote S3 storage

### Create metadata.yaml 

In metadata we can use DVC's hash to version deployed models

In [12]:
%%writefile metadata.yaml

name: iris
versions: [iris/dvc:8104914e6936da9864603b9bc4be2114]
platform: sklearn
inputs:
- datatype: BYTES
  name: input
  shape: [ 1, 4 ]
outputs:
- datatype: BYTES
  name: output
  shape: [ 3 ]

Overwriting metadata.yaml


### Create bucket for our trained model and push it

In [13]:
%%bash
mc mb minio-local/dvc-iris -p

mc cp model.joblib minio-local/dvc-iris/
mc cp metadata.yaml minio-local/dvc-iris/

Bucket created successfully `minio-local/dvc-iris`.
`model.joblib` -> `minio-local/dvc-iris/model.joblib`
Total: 0 B, Transferred: 1.05 KiB, Speed: 142.02 KiB/s
`metadata.yaml` -> `minio-local/dvc-iris/metadata.yaml`
Total: 0 B, Transferred: 199 B, Speed: 30.96 KiB/s


In [14]:
!mc ls minio-local/dvc-iris

[2020-04-21 12:23:41 BST]    199B metadata.yaml
[2020-04-21 12:23:41 BST]  1.1KiB model.joblib


## Deploy sklearn server

In [15]:
%%writefile secret.yaml

apiVersion: v1
kind: Secret
metadata:
  name: seldon-init-container-secret
type: Opaque
stringData:
  AWS_ACCESS_KEY_ID: minioadmin
  AWS_SECRET_ACCESS_KEY: minioadmin
  AWS_ENDPOINT_URL: http://minio.minio-system.svc.cluster.local:9000
  USE_SSL: "false"

Overwriting secret.yaml


In [16]:
!kubectl apply -f secret.yaml

secret/seldon-init-container-secret configured


In [17]:
%%writefile deploy.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: minio-dvc-sklearn
spec:
  annotations:
    seldon.io/executor: "true"
  name: iris
  predictors:
  - componentSpecs:
    graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: s3://dvc-iris
      envSecretRefName: seldon-init-container-secret
      name: classifier
    name: default
    replicas: 1

Overwriting deploy.yaml


In [18]:
!kubectl apply -f deploy.yaml

seldondeployment.machinelearning.seldon.io/minio-dvc-sklearn created


In [51]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=minio-dvc-sklearn -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "minio-dvc-sklearn-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "minio-dvc-sklearn-default-0-classifier" successfully rolled out


## Test deployment

### Test prediction

In [19]:
%%bash
curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://localhost:8003/seldon/seldon/minio-dvc-sklearn/api/v1.0/predictions  | jq .

{
  "data": {
    "names": [
      "t:0",
      "t:1",
      "t:2"
    ],
    "ndarray": [
      [
        0.9548873249364185,
        0.04505474761561256,
        5.792744796895459e-05
      ]
    ]
  },
  "meta": {}
}


### Test model metadata (optional)

In [20]:
%%bash
curl -s http://localhost:8003/seldon/seldon/minio-dvc-sklearn/api/v1.0/metadata/classifier | jq .

{
  "inputs": [
    {
      "datatype": "BYTES",
      "name": "input",
      "shape": [
        1,
        4
      ]
    }
  ],
  "name": "iris",
  "outputs": [
    {
      "datatype": "BYTES",
      "name": "output",
      "shape": [
        3
      ]
    }
  ],
  "platform": "sklearn",
  "versions": [
    "iris/dvc:8104914e6936da9864603b9bc4be2114"
  ]
}


## Cleanup

In [21]:
!kubectl delete -f deploy.yaml

seldondeployment.machinelearning.seldon.io "minio-dvc-sklearn" deleted


In [22]:
!rm .env -r